In [1]:
from pathlib import Path
import geopandas as gpd

In [2]:
pcon21 = gpd.read_file('../../data/reference/pcon.geojson')[["PCON21CD", "PCON21NM"]].set_index("PCON21NM")

The conventional names seem to have changed to omit a comma if a name of city included in constituency name. Need to clean these ip

In [3]:
pcon21.index = pcon21.index.str.replace(r'(Birmingham|Brighton|Ealing|Enfield|Lewisham|Liverpool|Manchester|Plymouth|Sheffield|Southampton),', r'\1', regex=True)

In [4]:
pcon24 = gpd.read_file('../../data/reference/uk-constituencies-2024.geojson')[["PCON24CD", "PCON24NM"]].set_index("PCON24NM")

In [5]:
changes = pcon21.join(
  pcon24,
  how="outer"
).sort_values('PCON24CD')
changes.index.names = ['PCON_NM']

In [6]:
data_dir = Path('../../data/reference/')

In [7]:
changes.to_csv(data_dir / "uk-constituencies-2024-changes.csv")

Let's take a look at the English changes

In [8]:
changes_e = changes[(changes.PCON21CD.str.startswith("E", na=False) | changes.PCON24CD.str.startswith("E", na=False))]

In [9]:
import pandas as pd
pd.Series(
    index=[
      'equivalence_found',
      'new',
      'retired',
      'count_21',
      'count_24',
    ],
    data=[
      (changes_e.PCON24CD.notna() & changes_e.PCON21CD.notna()).sum(),
      (changes_e.PCON24CD.notna() & changes_e.PCON21CD.isna()).sum(),
      (changes_e.PCON24CD.isna() & changes_e.PCON21CD.notna()).sum(),
      (changes_e.PCON21CD.notna()).sum(),
      (changes_e.PCON24CD.notna()).sum(),
    ]
)

equivalence_found    374
new                  169
retired              159
count_21             533
count_24             543
dtype: int64

In [10]:
changes_e[changes_e.PCON21CD.isna()]

,PCON21CD,PCON24CD
PCON_NM,,
Barnsley North,NaN,E14001074
Barnsley South,NaN,E14001075
Beckenham and Penge,NaN,E14001083
Bethnal Green and Stepney,NaN,E14001086
Bicester and Woodstock,NaN,E14001090
...,...,...
Wetherby and Easingwold,NaN,E14001582
Whitehaven and Workington,NaN,E14001583
Widnes and Halewood,NaN,E14001584


There has been a net increase of ten English constituencies, with two coming from Scotland and eight from Wales. Northern Irish constituency counts are the same.

In [11]:
pd.concat([
  pcon21.PCON21CD.str.slice(0, 1).reset_index().rename(columns={'PCON21CD': 'Nation'}).groupby('Nation').count(),
  pcon24.PCON24CD.str.slice(0, 1).reset_index().rename(columns={'PCON24CD': 'Nation'}).groupby('Nation').count()
], axis=1)

,PCON21NM,PCON24NM
Nation,,
E,533,543
N,18,18
S,59,57
W,40,32
